<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Transformer_Models_for_Time_Series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

class TimeSeriesTransformer(nn.Module):
    def __init__(self, num_features, num_heads, num_layers, dropout=0.1):
        super().__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=num_features, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(num_features, 1)

    def forward(self, src):
        output = self.transformer_encoder(src)
        output = self.fc(output)
        return output[-1]  # Take the last element of the output sequence

def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L - tw):
        train_seq = input_data[i:i + tw]
        train_label = input_data[i + tw:i + tw + 1]
        inout_seq.append((train_seq, train_label))
    return inout_seq

# Generate synthetic time series data
time_steps = np.linspace(0, np.pi * 2, 100)
data = np.sin(time_steps)

train_data_normalized = (data - np.mean(data)) / np.std(data)
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1, 1)  # Updated view for compatibility

train_window = 12
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)

model = TimeSeriesTransformer(num_features=1, num_heads=1, num_layers=1)  # Adjust num_features for compatibility
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

epochs = 150
for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()

        y_pred = model(seq.unsqueeze(1))

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i % 25 == 0:
        print(f'Epoch: {i:3} Loss: {single_loss.item():10.8f}')

# Predict the next values
test_inputs = train_data_normalized[-train_window:].tolist()
model.eval()
for i in range(12):
    # Ensure test_inputs is a list of lists
    print(f"test_inputs (length {len(test_inputs)}): {test_inputs[-train_window:]}")  # Debugging print
    seq = torch.FloatTensor(test_inputs[-train_window:]).view(-1, 1)
    with torch.no_grad():
        prediction = model(seq.unsqueeze(1)).item()
        test_inputs.append([prediction])  # Append as list

# Verify that test_inputs is a list of lists after the loop
if not all(isinstance(x, list) for x in test_inputs):
    print("Error: Not all elements in test_inputs are lists!")
else:
    print("Success: All elements in test_inputs are lists!")

plt.plot(data)
plt.plot(range(100, 112), [x[0] for x in test_inputs[train_window:]], 'r')
plt.show()